In [1]:
import pandas as pd
import geopandas as gpd 

# Vacant Property Dataset

In [2]:
df = pd.read_csv(r"C:\Users\Siddhant\Downloads\vacant_pins_with_block_group_cc_Flag.csv")

In [3]:
df.head()

,pin,longitude,latitude,is_inside_cc,GEOID
0,1011000040000,NaN,NaN,no,NaN
1,1011000060000,NaN,NaN,no,NaN
2,1011000090000,NaN,NaN,no,NaN
3,1011000370000,NaN,NaN,no,NaN
4,1011000710000,NaN,NaN,no,NaN


In [4]:
df['is_inside_cc'].value_counts()

is_inside_cc
no     21813
yes     7698
Name: count, dtype: int64

In [ ]:
# Group by GEOID and is_inside_cc, count the number of pins
grouped = df.groupby(['GEOID', 'is_inside_cc'])['pin'].count().unstack(fill_value=0).reset_index()

# Optional: Rename the columns for clarity
grouped = grouped.rename(columns={'yes': 'inside_cc', 'no': 'not_inide_cc'})


In [10]:
grouped.head()

is_inside_cc,GEOID,not_inide_cc,inside_cc
0,1.703101e+11,2,0
1,1.703101e+11,0,1
2,1.703101e+11,1,1
3,1.703101e+11,2,0
4,1.703101e+11,1,0


In [9]:
grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2605 entries, 0 to 2604
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   GEOID         2605 non-null   float64
 1   not_inide_cc  2605 non-null   int64  
 2   inside_cc     2605 non-null   int64  
dtypes: float64(1), int64(2)
memory usage: 61.2 KB


In [21]:
grouped['GEOID'] = grouped['GEOID'].astype(int)

In [12]:
grouped.head()

is_inside_cc,GEOID,not_inide_cc,inside_cc
0,170310101002,2,0
1,170310101003,0,1
2,170310102011,1,1
3,170310102013,2,0
4,170310102021,1,0


In [15]:
grouped.isnull().sum()

is_inside_cc
GEOID           0
not_inide_cc    0
inside_cc       0
dtype: int64

In [22]:
grouped['GEOID'] = grouped['GEOID'].astype(str)

# Chicago Block Group Dataset

In [17]:
gdf = gpd.read_file(r"E:\IIT Chicago\Sem 4\Data Science Practicum - CSP-572\Analysis\Dataset Files\blkgrp_data_with ethnicity_data.geojson")

In [18]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3950 entries, 0 to 3949
Data columns (total 37 columns):
 #   Column                                                                          Non-Null Count  Dtype   
---  ------                                                                          --------------  -----   
 0   vacant_count                                                                    3950 non-null   float64 
 1   blkgrp                                                                          3950 non-null   object  
 2   population                                                                      3950 non-null   float64 
 3   households                                                                      3950 non-null   float64 
 4   median_income                                                                   3950 non-null   float64 
 5   housing_units                                                                   3950 non-null   float64 
 6   

In [19]:
gdf.drop('vacant_count', axis = 1, inplace = True)

In [24]:
# Merge all columns from df into gdf on GEOID = blkgrp
gdf = gdf.merge(grouped, left_on='blkgrp', right_on='GEOID', how='left')

# Optionally drop the extra 'GEOID' column from df if not needed
gdf.drop(columns='GEOID', inplace=True)

In [25]:
gdf.head(3)

,blkgrp,population,households,median_income,housing_units,avg_VMT_per_car,median_VMT_per_car,avg_MPG_per_car,median_MPG_per_car,avg_Emissions_per_car,...,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,Two or More Races,Two or More Races_Two races including Some Other Race,"Two or More Races_Two races excluding Some Other Race, and three or more races",geometry,not_inide_cc,inside_cc
0,170312909001,639.0,301.0,0.0,370.0,13000.000000,9000.0,22.736364,22.0,4.859271,...,0,0,0,0,19,19,0,"POLYGON ((-87.73006 41.8661, -87.72981 41.8661...",26.0,4.0
1,170312909001,639.0,301.0,0.0,370.0,13000.000000,9000.0,22.736364,22.0,4.859271,...,0,0,0,0,19,19,0,"POLYGON ((-87.73006 41.8661, -87.72981 41.8661...",26.0,4.0
2,170312909002,1252.0,405.0,21708.0,493.0,23272.727273,14000.0,22.547619,22.0,10.220586,...,0,0,0,0,0,0,0,"POLYGON ((-87.72995 41.86245, -87.7297 41.8624...",22.0,10.0


In [3]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3950 entries, 0 to 3949
Data columns (total 38 columns):
 #   Column                                                                          Non-Null Count  Dtype   
---  ------                                                                          --------------  -----   
 0   blkgrp                                                                          3950 non-null   object  
 1   population                                                                      3950 non-null   float64 
 2   households                                                                      3950 non-null   float64 
 3   median_income                                                                   3950 non-null   float64 
 4   housing_units                                                                   3950 non-null   float64 
 5   avg_VMT_per_car                                                                 3950 non-null   float64 
 6   

In [4]:
for col in gdf.select_dtypes(include='object').columns:
    if col != 'blkgrp':
        gdf[col] = gdf[col].astype(float).fillna(0).astype(int)

In [9]:
gdf.rename(columns={'not_inide_cc': 'not_inside_cc'}, inplace=True)

In [10]:
gdf['total_vp_count'] = gdf['inside_cc'] + gdf['not_inside_cc']

In [11]:
gdf.head(3)

,blkgrp,population,households,median_income,housing_units,avg_VMT_per_car,median_VMT_per_car,avg_MPG_per_car,median_MPG_per_car,avg_Emissions_per_car,...,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,Two or More Races,Two or More Races_Two races including Some Other Race,"Two or More Races_Two races excluding Some Other Race, and three or more races",not_inside_cc,inside_cc,geometry,total_vp_count
0,170312909001,639.0,301.0,0.0,370.0,13000.000000,9000.0,22.736364,22.0,4.859271,...,0,0,0,19,19,0,26,4,"POLYGON ((-87.73006 41.8661, -87.72981 41.8661...",30
1,170312909001,639.0,301.0,0.0,370.0,13000.000000,9000.0,22.736364,22.0,4.859271,...,0,0,0,19,19,0,26,4,"POLYGON ((-87.73006 41.8661, -87.72981 41.8661...",30
2,170312909002,1252.0,405.0,21708.0,493.0,23272.727273,14000.0,22.547619,22.0,10.220586,...,0,0,0,0,0,0,22,10,"POLYGON ((-87.72995 41.86245, -87.7297 41.8624...",32


In [12]:
gdf.to_file(r"E:\IIT Chicago\Sem 4\Data Science Practicum - CSP-572\Analysis\Dataset Files\blkgrp_data_with vacant_property_data.geojson", driver="GeoJSON")